In [ ]:
!pip install flask --break-system-packages
!pip install flask-restx  --break-system-packages

## **Example API System**

### **RESTful API Features**

#### **1. General Greeting**
- **GET `/example/hello`**
  - **Description**: Returns a generic greeting message.
  - **Response Schema/Model**:
    - `message`: string 
  - **HTTP Status Codes**:
    - `200 OK`: Successful response.

#### **2. Personalized Greeting**
- **GET `/example/greet/<string:name>`**
  - **Description**: Returns a personalized greeting message using the `name` path parameter.
  - **Path Parameters**:
    - `name` (string): The name of the person to greet.
  - **Response Schema/Model**: 
    - `name`: string,
    - `message`: string 
  - **HTTP Status Codes**:
    - `200 OK`: Successful response.

#### **3. Query-Based Greeting**
- **GET `/example/query`**
  - **Description**: Returns a greeting message based on query parameters (`name` and `age`).
  - **Query Parameters**:
    - `name` (string, optional): Name of the person. Defaults to `"Guest"`.
    - `age` (integer, optional): Age of the person.
  - **Response Schema/Model**:  
    - `name` (string, optional): Name of the person. Defaults to `"Guest"`.
    - `age` (integer, optional): Age of the person.
    - `message`: str, 
  - **HTTP Status Codes**:
    - `200 OK`: Successful response.
    - `400 Bad Request`: Invalid query parameters.
---

 


In [8]:
import nest_asyncio
from flask import Flask, request
from flask_restx import Api, Resource, fields
from werkzeug.serving import run_simple

nest_asyncio.apply()

# initialize Flask app and FLASK RESTX API
app = Flask(__name__)
api = Api(app, version="1.0", title="Flask RESTX API", description="A demonstration of Flask-RESTX with class-based routes")

example_ns = api.namespace('example', description='Endpoints demonstrating basic Flask-RESTX functionality')

# model has two uses 
# 1. for marshaling to make the response network transportable
# 2. for validating checks if the response matches the model

# hello endpoint response model 
hello_model = example_ns.model('hello', {
    "message": fields.String(description="Greeting message", example="Hello, World!")
})

# greet endpoint response model
greet_model = example_ns.model('Greet', {
    'name': fields.String(required=True, description="Name of the person being greeted", example="Adam"),
    'message': fields.String(description="Personalized greeting message", example="Hello, Adam!")
})

# query endpoint response model
query_model = example_ns.model('Query', {
    'name': fields.String(description="Name parameter from query string", example="Adam"),
    'age': fields.Integer(description="Age parameter from query string", example=30),
    'message': fields.String(description="Response message", example="Hello, Adam! You are 30 years old.")
})

@example_ns.route('/hello')
class HelloWorld(Resource):
  
    @example_ns.marshal_with(hello_model)
    def get(self):
        """Returns a generic greeting message."""
        
        return {"message": " Hello, world"}, 200
        
 
@example_ns.route('/greet/<string:name>')
@example_ns.param('name', description='Name of the person to greet')
class Greet(Resource):
    
    @example_ns.marshal_with(greet_model)
    def get(self, name: str):
        """Returns a personalized greeting message using the `name` path parameter."""
        
        return {"name": name, "message": f"Hello {name}!"}

# http://localhost:5000/example/query?name="adam"&age=30
@example_ns.route('/query')
class Query(Resource):
    @example_ns.doc(params={
     'name': {'description': 'Name parameter (optional)', 'default': 'Guest'},
     'age': {'description': 'Age parameter (optional)', 'type': 'integer'}
    })
    @example_ns.marshal_with(query_model)
    def get(self):
        """Returns a greeting message based on query parameters (`name` and `age`)."""
        name = request.args.get("name", "Guest")
        age = request.args.get("age")
        
        
        # Check if 'age' is provided and construct the message accordingly
        if age:
            return {"name": name, "age": int(age), "message": f"Hello, {name}! You are {age} years old."}, 200

        return {"name": name, "message": f"Hello, {name}!"}, 200



api.add_namespace(example_ns)

# Run  Flask in Jupyter Notebook 
run_simple('localhost', 5000, app)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jan/2025 20:56:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2025 20:56:49] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Jan/2025 20:56:49] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Jan/2025 20:56:49] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Jan/2025 20:56:49] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Jan/2025 20:56:50] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2025 20:56:50] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 304 -
127.0.0.1 - - [03/Jan/2025 20:59:25] "GET /example/query?name=adam&age=343 HTTP/1.1" 200 -
